# Collecting the data from GCloud

In [1]:

import pandas as pd
import pandas_gbq

from google.oauth2 import service_account

key_path = "./app/gkeys/epidemicapp-62d0d471b86f.json"
CREDENTIALS  = service_account.Credentials.from_service_account_file(key_path)
pandas_gbq.context.credentials = CREDENTIALS


In [2]:
query_data = """
    select *
    from br_general.cities_coordinates
"""

cities_df = pandas_gbq.read_gbq(query_data, project_id="epidemicapp-280600")
cities_df.head()

Downloading: 100%|██████████| 2540/2540 [00:01<00:00, 1521.69rows/s]


,city,state,country,lat,long
0,Rio Branco/AC,AC,Brasil,-9.976536,-67.822078
1,Plácido de Castro/AC,AC,Brasil,-10.323915,-67.182420
2,Cruzeiro do Sul/AC,AC,Brasil,-7.630796,-72.670387
3,Acrelândia/AC,AC,Brasil,-10.075917,-67.052690
4,Senador Guiomard/AC,AC,Brasil,-9.976500,-67.319140


## Getting the cities IBGE identifiers

In [3]:

import requests
import json

data = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/municipios").json()
city_df = pd.DataFrame()

for city in data:
    city_id = city["id"]
    city_name = city["nome"]
    city_state = city["microrregiao"]["mesorregiao"]["UF"]["sigla"]
    city_state_id = city["microrregiao"]["mesorregiao"]["UF"]["id"]
    
    new_df = pd.DataFrame({
        "id": [int(city_id)],
        "city": [city_name],
        "state": [city_state]})
    city_df = pd.concat((city_df, new_df))

In [4]:
cities_df["city_name"] = cities_df["city"]
cities_df["city"] = [c[:-3] for c in cities_df["city"].tolist()]

In [5]:
joined_df = pd.merge(cities_df, city_df, how='left', on=["city", "state"])
city_df = joined_df.drop_duplicates(subset=["city"]).dropna(subset=["id"])

In [6]:
city_df.head()

,city,state,country,lat,long,city_name,id
0,Rio Branco,AC,Brasil,-9.976536,-67.822078,Rio Branco/AC,1200401.0
1,Plácido de Castro,AC,Brasil,-10.323915,-67.182420,Plácido de Castro/AC,1200385.0
2,Cruzeiro do Sul,AC,Brasil,-7.630796,-72.670387,Cruzeiro do Sul/AC,1200203.0
3,Acrelândia,AC,Brasil,-10.075917,-67.052690,Acrelândia/AC,1200013.0
4,Senador Guiomard,AC,Brasil,-9.976500,-67.319140,Senador Guiomard/AC,1200450.0


## Upload the cities locations and identifiers to GCloud 

In [8]:

pandas_gbq.to_gbq(city_df[["id", "city", "state", "country", "city_name"]], 
                  "br_general.identifier", 
                  project_id="epidemicapp-280600", 
                  credentials=CREDENTIALS, 
                  if_exists="replace")

pandas_gbq.to_gbq(city_df[["id", "lat", "long"]], 
                  "br_general.locations", 
                  project_id="epidemicapp-280600", 
                  credentials=CREDENTIALS, 
                  if_exists="replace")


1it [00:03,  3.96s/it]
1it [00:04,  4.30s/it]


# Obtaining the social indicators

            

## Creating the full content dataframe

In [7]:
full_content = pd.DataFrame()

In [33]:

import numpy as np
from tqdm import tqdm
from progress.bar import FillingCirclesBar

def query_indicator(city_id=None, indicator_name="indicator", indicator=None, research=None, operation="mean"):
    ibge_url = "https://servicodados.ibge.gov.br/api/v1/pesquisas/{}/periodos/all/indicadores/{}/resultados/{}"
    ind_df = pd.DataFrame()
    bar = FillingCirclesBar('Processing', max=len(city_id))
    for ident in tqdm(city_id):
        try:
            data = requests.get(ibge_url.format(research, indicator, int(ident))).json()
            request_flag = True
        except Exception as e:
            #print("Get request exception at: {}".format(int(ident)))
            request_flag = False
        if request_flag and len(data) > 0:
            ind_list = []
            for content in data[0]["res"]:
                years_list = list(content["res"].keys())
                years_full = list()
                for y in years_list:
                    if content["res"][y] != None:
                        years_full.append(y)
                years_list = [int(y) for y in years_full]
                try:
                    key = str(max(years_list))
                    ind_value = float(content["res"][key])
                    ind_list.append(ind_value)
                except Exception as e:
                    pass #print("Error at {} => {}".format(int(ident), e))
        
            new_df = pd.DataFrame({
                indicator_name: np.mean(ind_list),
                "city_id": [int(ident)],
                "id": [int(ident)]
            })
            ind_df = pd.concat((ind_df, new_df))
    return ind_df
        

## IDH Indice de Desenvolvimento Humano


In [ ]:

# IBGE API URL for IDH indicator
ibge_url = "https://servicodados.ibge.gov.br/api/v1/pesquisas/23/periodos/all/indicadores/30255/resultados/{}"

idh_df = pd.DataFrame()
for city_id in city_df["id"].tolist():
    
    data = requests.get(ibge_url.format(int(city_id))).json()
    
    idh_list = []
    for content in data[0]["res"]:
        
        years_list = content["res"].keys()
        years_list = [int(y) for y in years_list]
        
        try:
            idh_value = float(content["res"][str(max(years_list))])
            idh_list.append(idh_value)
        except Exception as e:
            print("Error at {} => {}".format(int(city_id), e))
    
    new_df = pd.DataFrame({
        "city_id": [int(city_id)]*len(idh_list),
        "idh": idh_list})
    idh_df = pd.concat((idh_df, new_df))


## Several other indicators


In [9]:

# IDH - Indice de Desenvolvimento Humano
idh_df = query_indicator(city_id=city_df["id"].tolist(), indicator_name="idh", indicator=30255, research=23)
idh_df.head()

 49%|████▉     | 1215/2459 [06:32<06:37,  3.13it/s]/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 2459/2459 [13:30<00:00,  3.03it/s]


,idh,city_id,id
0,0.727,1200401,1200401
0,0.622,1200385,1200385
0,0.664,1200203,1200203
0,0.604,1200013,1200013
0,0.640,1200450,1200450


In [12]:

# PIB per capita
pib_df = query_indicator(city_id=city_df["id"].tolist(), indicator_name="pib", indicator=47001, research=38)
pib_df.head()


100%|██████████| 2459/2459 [14:05<00:00,  2.91it/s]


,pib,city_id,id
0,21258.68,1200401,1200401
0,13030.70,1200385,1200385
0,15585.70,1200203,1200203
0,15984.09,1200013,1200013
0,16692.58,1200450,1200450


In [17]:

# Indice de desenvolvimento de educação básica - Inicial
ide_bi_df = query_indicator(city_id=city_df["id"].tolist(), indicator_name="ide_bi", indicator=78187, research=40)
ide_bi_df.head()

# Indice de desenvolvimento de educação básica - Final
ide_bf_df = query_indicator(city_id=city_df["id"].tolist(), indicator_name="ide_bf", indicator=78192, research=40)
ide_bf_df.head()


100%|██████████| 2459/2459 [14:21<00:00,  2.85it/s] 


,ide_bf,city_id,id
0,4.8,1200401,1200401
0,4.8,1200385,1200385
0,4.7,1200203,1200203
0,4.5,1200013,1200013
0,5.0,1200450,1200450


In [18]:
# Censo - População residente
ce_pr_df = query_indicator(city_id=city_df["id"].tolist(), indicator_name="pop_residente", indicator=30281, research=43)
ce_pr_df.head()

# Censo - Residencias
ce_re_df = query_indicator(city_id=city_df["id"].tolist(), indicator_name="num_residencias", indicator=30282, research=43)
ce_re_df.head()


100%|██████████| 2459/2459 [14:05<00:00,  2.91it/s]


,num_residencias,city_id,id
0,94184.0,1200401,1200401
0,4748.0,1200385,1200385
0,18573.0,1200203,1200203
0,3459.0,1200013,1200013
0,5589.0,1200450,1200450


In [34]:
# Densidade demográfica
dens_dem_df = query_indicator(city_id=city_df["id"].tolist(), indicator_name="dens_demografica", indicator=29168, research=33)
dens_dem_df.head()


100%|██████████| 2459/2459 [13:12<00:00,  3.10it/s]


,dens_demografica,city_id,id
0,38.03,1200401,1200401
0,8.86,1200385,1200385
0,8.94,1200203,1200203
0,6.94,1200013,1200013
0,8.69,1200450,1200450


# Upload data to GCloud

In [35]:

pandas_gbq.to_gbq(idh_df, "br_general.idh", 
                  project_id="epidemicapp-280600", credentials=CREDENTIALS, if_exists="replace")
pandas_gbq.to_gbq(pib_df, "br_general.pib", 
                  project_id="epidemicapp-280600", credentials=CREDENTIALS, if_exists="replace")
pandas_gbq.to_gbq(ide_bi_df, "br_general.idebi", 
                  project_id="epidemicapp-280600", credentials=CREDENTIALS, if_exists="replace")
pandas_gbq.to_gbq(ide_bf_df, "br_general.idebf", 
                  project_id="epidemicapp-280600", credentials=CREDENTIALS, if_exists="replace")
pandas_gbq.to_gbq(ce_pr_df, "br_general.censo_pop_res", 
                  project_id="epidemicapp-280600", credentials=CREDENTIALS, if_exists="replace")
pandas_gbq.to_gbq(ce_re_df, "br_general.censo_resid", 
                  project_id="epidemicapp-280600", credentials=CREDENTIALS, if_exists="replace")
pandas_gbq.to_gbq(dens_dem_df, "br_general.dens_dem", 
                  project_id="epidemicapp-280600", credentials=CREDENTIALS, if_exists="replace")


1it [00:07,  7.80s/it]
1it [00:06,  6.01s/it]
1it [00:05,  5.03s/it]
1it [00:08,  8.70s/it]
1it [00:05,  5.51s/it]
1it [00:08,  8.88s/it]
1it [00:04,  4.85s/it]
